# Análise de Respostas Médias

Este notebook lê os arquivos `*_avg.csv` gerados pela simulação, plota as respostas média para cada combinação de parâmetros (α, β, δ), calcula métricas de controle (Mp, Tr e Ts), seleciona as 10 melhores combinações e plota essas respostas para análise.

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Função para ler arquivos `_avg.csv`
Os arquivos têm linhas com 8 valores de estado e objetivo, seguidos de tempo e número de trial.

In [ ]:
def read_avg_csv(filepath):
    t_list = []
    y_list = []
    with open(filepath, 'r') as f:
        lines = f.readlines()
    for line in lines[1:]:  # pula cabeçalho
        if not line.strip():
            continue
        parts = line.strip().split()
        if len(parts) < 10:
            continue
        x_real = [float(v) for v in parts[0:4]]
        t = float(parts[8])
        t_list.append(t)
        y_list.append(x_real[0])  # somente o primeiro estado
    return np.array(t_list), np.array(y_list)

## 2. Leitura dos arquivos `_avg.csv` e extração de parâmetros

In [ ]:
# Ajuste o diretório se necessário
directory = '.'

# Localiza arquivos de média
files = glob.glob(os.path.join(directory, '*_avg.csv'))

# Regex para extrair α (mili), β (micro) e δ (mili)
pattern = re.compile(r'A(?P<alpha>\d+)m_B(?P<beta>\d+)u_D(?P<delta>\d+)m_avg\.csv')

data = []
for f in sorted(files):
    m = pattern.match(os.path.basename(f))
    if not m:
        continue
    alpha = int(m.group('alpha')) / 1000.0
    beta = int(m.group('beta')) / 1e6
    delta = int(m.group('delta')) / 1000.0
    t, y = read_avg_csv(f)
    data.append({'alpha': alpha, 'beta': beta, 'delta': delta, 't': t, 'y': y})
print(f'Carregadas {len(data)} respostas médias.')

## 3. Plot de todas as respostas médias

In [ ]:
plt.figure(figsize=(10, 6))
for entry in data:
    label = f"α={entry['alpha']}, β={entry['beta']}, δ={entry['delta']}"
    plt.plot(entry['t'], entry['y'], label=label)
plt.title('Respostas Médias - Todas Combinações')
plt.xlabel('Tempo (s)')
plt.ylabel('Saída (Estado 0)')
plt.legend(fontsize='small', ncol=2)
plt.show()

## 4. Cálculo de métricas de desempenho (Mp, Tr, Ts)

In [ ]:
def compute_metrics(t, y):
    y_final = y[-1]
    Mp = (np.max(y) - y_final) / y_final * 100 if y_final != 0 else np.nan
    # Rise time entre 10% e 90%
    y10 = 0.1 * y_final
    y90 = 0.9 * y_final
    try:
        t10 = t[np.where(y >= y10)[0][0]]
        t90 = t[np.where(y >= y90)[0][0]]
        Tr = t90 - t10
    except IndexError:
        Tr = np.nan
    # Settling time ±2%
    tol = 0.02 * y_final
    idx = np.where(np.abs(y - y_final) > tol)[0]
    Ts = t[idx[-1]] if len(idx) > 0 else 0.0
    return Mp, Tr, Ts

## 5. Cálculo de métricas para cada combinação

In [ ]:
metrics = []
for entry in data:
    Mp, Tr, Ts = compute_metrics(entry['t'], entry['y'])
    metrics.append({'alpha': entry['alpha'], 'beta': entry['beta'], 'delta': entry['delta'], 'Mp': Mp, 'Tr': Tr, 'Ts': Ts})
df_metrics = pd.DataFrame(metrics)
df_metrics

## 6. Seleção das 10 melhores combinações
Critério: soma dos ranks de Mp, Tr e Ts (menor é melhor)

In [ ]:
df_rank = df_metrics.copy()
for col in ['Mp', 'Tr', 'Ts']:
    df_rank[f'{col}_rank'] = df_rank[col].rank(method='min')
df_rank['rank_sum'] = df_rank[['Mp_rank', 'Tr_rank', 'Ts_rank']].sum(axis=1)
df_best = df_rank.nsmallest(10, 'rank_sum')
df_best

## 7. Plot das 10 melhores respostas

In [ ]:
plt.figure(figsize=(10, 6))
for _, row in df_best.iterrows():
    e = next(e for e in data if (e['alpha'] == row['alpha'] and e['beta'] == row['beta'] and e['delta'] == row['delta']))
    label = f"α={row['alpha']}, β={row['beta']}, δ={row['delta']}"
    plt.plot(e['t'], e['y'], label=label)
plt.title('Respostas Médias - Top 10 Combinações')
plt.xlabel('Tempo (s)')
plt.ylabel('Saída (Estado 0)')
plt.legend(fontsize='small')
plt.show()